# 1. 서비스 소개 및 분석 방향
## 1-1. 서비스 소개
### 음화당 : 음악을 좋아하는 당신께, 이 영화를 드려요.
영화 타이틀곡의 음악적 요소를 기반으로 데이터 분석을 하여 영화를 추천해주는, 음악적 요소 기반 영화 추천 서비스 입니다.
평소에 음악을 즐겨 듣는, 특히 영화에서의 음악을 중시하는 사용자를 위한 서비스입니다.

코로나로 인해 부정적인 감정이 많이 증가됐다는 통계가 있습니다. 스트레스를 받을 때 음악을 많이 듣는다는 연구결과에서 착안해 유저의 기분과 듣고 싶은 음악을 기준으로 영화를 추천해주는 서비스를 구현하게 되었습니다.

유저는 다섯 가지의 서로 다른 분위기의 영화 타이틀곡 중에 하나를 선택하게 되며, 저희는 그 타이틀곡들이 속해있는 영화의 장르로 데이터베이스를 한번 필터링 합니다.
그 다음, 유저가 현재 느끼는 감정 (혹은 유저가 원하는 무드)과 음악의 템포를 입력하게 되면, 저희는 그것을 기반으로 데이터베이스를 한번 더 필터링 합니다. 
음악적 요소들은 다음과 같습니다: energy, danceability, valence, tempo
유저는 필터링된 영화 음악속에서 1개를 선택하게 됩니다. 저희는 데이터 분석을 통해 그 영화 음악과 비슷한 음악을 가진 영화를 추천합니다.
데이터 분석에 사용되는 음악적 요소들은 다음과 같습니다 : acousticness, danceability, energy, tempo, valence, instrumentalness, liveness, loudness, speechiness

## 1-2. 분석 방향
영화가 장르로 구분이 되는 것처럼 영화 음악 또한 비슷한 음악끼리 묶일 수도 있다는 가설을 설정하고 그에 맞게 진행하였습니다. 
군집 형성을 보기 위해 **K-means clustering**을 사용하고 군집의 개수는 **Elbow method**를 통해 4개로 결정하였습니다. clustering은 정규화된 영화 음악의 feature들을 기준으로 했습니다. **PCA**와 **t-sne**로 시각화 하여 군집이 잘 형성됐는지 확인하였습니다.
유저가 선택한 영화 음악과 비슷한 음악을 찾기 위해 같은 cluster내에서 **Euclidean distance**를 이용하였습니다.

## 1-3. Data 추출 및 정제
저희는 [Movies on OTT platforms](https://www.kaggle.com/javagarm/movies-on-ott-platforms) data를 기준으로 진행하였습니다. 이 데이터 외에 필요한 data들은 웹 크롤링을 이용해 추출하였습니다.
- 추가로 필요한 data : 영화 음악 감독, 영화 포스터, 영화 음악 features, 영화 줄거리 등
- https://www.soundtrack.net/ 사이트에서 영화 음악 감독과 영화 poster link crawling을 진행하였습니다.
- [Spotify Web api](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features)를 이용해 영화 음악의 features를 추출하였습니다.
- [네이버 영화](https://movie.naver.com/)에서 영화 줄거리 추출하였습니다.

# 2. Data Crawling
## 2-1. 영화 음악 감독 crawling

In [1]:
import pandas as pd

data = pd.read_csv('movie.csv') # 원 데이터
movie = data
movie.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,87%,1.0,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1.0,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1.0,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1.0,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1.0,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [5]:
# title 정제
lower_title = movie['Title'].str.lower()
title_re1 = lower_title.str.replace(pat=r'[^\w\']', repl=r' ', regex=True) # 특수문자 제거("'" 빼고)
title_re2 = title_re1.str.replace(pat="'", repl="", regex=False) 
title_re3 = title_re2.str.replace('  ',' ').str.replace(' ','-') # 공백 -> '-'
fin_title = title_re3.str.replace(pat=r"['-'$]", repl=r'', regex=True).str.replace('--','-')
movie['re_title'] = fin_title
movie['re_title'].head()

0                        inception
1                       the-matrix
2            avengers-infinity-war
3               back-to-the-future
4    the-good-the-bad-and-the-ugly
Name: re_title, dtype: object

In [ ]:
import re
import requests
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

for i in movie['re_title']:
    if type(i) == str and i[-1] == '-':
        try:
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('https://', adapter)

            url = f'https://www.soundtrack.net/movie/{i[:-1]}/'
            html = session.get(url, timeout=120).text
            time.sleep(random.uniform(2,4))
        except HTTPError as e:
            continue
        else:
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.find('li', {'class':'list-group-item'})
            if not text:
                continue
            elif 'composer' in text.get_text():
                cp = re.search('composer', text.get_text()).end()
                movie.loc[movie['re_title'] == i, 'music_director'] = text.get_text()[cp:]
            elif 'music supervisor' in text.get_text():
                mv = re.search('music supervisor', text.get_text()).end()
                movie.loc[movie['re_title'] == i, 'music_director'] = text.get_text()[mv:]

## 2-2. 영화 poster link crawling

In [ ]:
import re
import requests
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

for i in movie['re_title']:
    try:
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('https://', adapter)

        url = f'https://www.soundtrack.net/movie/{i}/'
        html = session.get(url, timeout=120).text
        time.sleep(random.uniform(2,4))
    except HTTPError as e:
        continue
    else:
        soup = BeautifulSoup(html, 'html.parser')
        text = soup.find('div', {'id':'titlebox'})
        if text:
            img = text.find('img')
            if img:
                movie.loc[movie['re_title'] == i, 'poster'] = f"https://www.soundtrack.net{img['src']}"

## 2-3. 영화 음악 feature crawling
1. spotify에서 영화 음악 감독을 검색하고 나온 앨범들에서 영화 이름을 찾아서 앨범 url 추출
2. 앨범에서 popularity 가장 높은 곡 track url 추출
3. track에서 음악 features 추출

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pprint
import csv
import pandas as pd
import numpy as np

CLIENT_ID = "{spotify_client_id}"
CLIENT_SECRET = "{spotify_client_secret}"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

In [8]:
# csv파일에서 영화 데이터 불러오기
info = pd.read_csv('fin_movie.csv')
info = info.drop(columns=['Unnamed: 0'])
info.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,re_title,music_director,poster,in_spe
0,1,Inception,2010,13+,8.8,87%,1.0,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,inception,Hans Zimmer,https://www.soundtrack.net/img/movie/30396.jpg,False
1,2,The Matrix,1999,18+,8.7,87%,1.0,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,the-matrix,Don Davis,https://www.soundtrack.net/img/movie/16705.jpg,False
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1.0,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,avengers-infinity-war,Alan Silvestri,https://www.soundtrack.net/img/movie/41685.jpg,False
3,4,Back to the Future,1985,7+,8.5,96%,1.0,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,back-to-the-future,Alan Silvestri,https://www.soundtrack.net/img/movie/1695.jpg,False
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1.0,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,the-good-the-bad-and-the-ugly,Ennio Morricone,https://www.soundtrack.net/img/movie/10072.jpg,False


In [ ]:
# 영화 정보 중 음악감독 정보를 중복값 제거하여 return
seen = set()
directors = []
for index, row in info.iterrows():
    name = row['Directors']
    if name=='':
        pass
    elif name not in seen:
        seen.add(name)
        directors.append(name)

In [ ]:
# 영화 음악 감독 id 추출
ids = []
for director in directors:
    results = sp.search(q=director, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        ids.append(items[0])
    else:
        pass

In [ ]:
# artist_set -> dictionary 변환
artists = {}
for _, row in artist_set.iterrows():
    artists[row['music_director']] = row['id']

In [ ]:
# song + artists
for idx, row in song.iterrows():
    if row['music_director'] not in artists:
        continue
    song.loc[idx, 'artist_id'] = artists[row['music_director']]

In [ ]:
# 영화 감독 id로 앨범 추출
for idx, row in song.loc[kor_idx, :].iterrows():
    if pd.isna(row['artist_id']):
        continue
    items = sp.artist_albums(row['artist_id'], album_type='album', limit=50, offset=250)['items']
    if len(items) < 1:
        continue
    for item in items:
        if row['title'] in item['name']:
            song.loc[idx, 'album_uri'] = item['uri']
            break

In [ ]:
# artist id를 통해 앨범 정보 가져오기
albums = []
album_list = []
seen = set()

results = sp.artist_albums('3eCpCUtYdBCM1twAb4mk8I', album_type='album')
albums.extend(results['items'])
pprint.pprint(albums)
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

albums.sort(key=lambda album: album['name'].lower())
for album in albums:
    name = album['name']
    id = album['id']
    if name not in seen:
        seen.add(name)
        name_id_dic = {}
        name_id_dic['name'] = name
        name_id_dic['id'] = id
        album_list.append(name_id_dic)
return (album_list)

In [ ]:
# spotify에서 영화 음악 앨범 urn 추출 (search를 통해서)
for idx, row in song.loc[kor_idx, :].iterrows():
    try:
        word = row['title'] + ' ' + row['music_director']
        album = sp.search(q=word, type='album')
        items = album['albums']['items'][0]
    except IndexError:
        continue
    else:
        for i in items['artists']:
            artist = i['name']
            title = items['name']
            if (artist == row['music_director']) and (row['title'] in title):
                song.loc[idx, 'album_uri'] = items['uri']
                break

In [ ]:
# 앨범에서 popularity가 높은 곡 추출
for index, row in song.iterrows():
    if row['album_uri']!=0:
        tracks = sp.album_tracks(row['album_uri'])
        tracks = tracks['items']
        ids = {}
        for track in tracks:
            track_pop = sp.track(track['id'])['popularity']
            track_id = sp.track(track['id'])['id']
            ids[track_id] = track_pop

        ids = sorted(ids.items(), key=lambda x: x[1], reverse=True)
        ids.append(ids[0])
        print(index,ids[0])
        song.loc[index,['most_popular']] = ids[0][0]
        song.loc[index,['popularity']] = ids[0][1]

In [ ]:
# popularity가 높은 곡의 url 추출
for idx, urn in enumerate(song[song['album_urn']]['album_urn']):
    tracks = sp.album_tracks(urn)
    tracks = tracks['items']
    uris = []
    for track in tracks:
        track_pop = sp.track(track['id'])['popularity']
        track_uri = sp.track(track['id'])['uri']
        uris.append((track_pop, track_uri))
    uris = sorted(uris, key=lambda x: x[0], reverse=True)
    if uris[0][0] == 0:
        continue
    song.loc[idx, 'track_uri'] = uris[0][1]

In [ ]:
# track의 feature 추출
for index, row in song.iterrows():
    try:
        features = sp.audio_features(row['most_popular'])
        features = features[0]
        song.loc[index,['acousticness']] = features['acousticness']
        song.loc[index,['danceability']] = features['danceability']
        song.loc[index,['energy']] = features['energy']
        song.loc[index,['tempo']] = features['tempo']
        song.loc[index,['valence']] = features['valence']
        song.loc[index,['instrumentalness']] = features['instrumentalness']
        song.loc[index,['liveness']] = features['liveness']
        song.loc[index,['loudness']] = features['loudness']
        song.loc[index,['speechiness']] = features['speechiness']
    except:
        song.loc[index,['acousticness']] = 0
        song.loc[index,['danceability']] = 0
        song.loc[index,['energy']] = 0
        song.loc[index,['tempo']] = 0
        song.loc[index,['valence']] = 0
        song.loc[index,['instrumentalness']] = 0
        song.loc[index,['liveness']] = 0
        song.loc[index,['loudness']] = 0
        song.loc[index,['speechiness']] = 0

In [ ]:
# 장르 분리
genre = {}
for i in movie['Genres']:
    if type(i) != str:
        continue
    for j in i.split(','):
        if j in genre:
            genre[j] += 1
        else:
            genre[j] = 1

## 2-4. 영화 페이지 url & 줄거리 crawling
- [네이버 영화](https://movie.naver.com/)에서 영화를 검색하고 영화 이름과 년도가 같은 영화의 페이지 url을 crawling
- 영화 페이지 url에서 줄거리 crawling

In [ ]:
# 영화 페이지 url crawling
for idx, row in movie_info[movie_info['movie_url'].isnull()].iterrows():
    print(idx)
    try:
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('https://', adapter)

        url = f"https://movie.naver.com/movie/search/result.naver?query={row['query_title']}"
        html = session.get(url, timeout=120).text
        time.sleep(random.uniform(2,4))
    except HTTPError as e:
        print('not page', i)
        continue
    else:
        soup = BeautifulSoup(html, 'html.parser')
        search_result = soup.find('ul', {'class':'search_list_1'})
        if search_result == None:
            continue
        results = search_result.findAll('li')
        
        if len(results) < 1:
            continue
        for result in results:
            result_name = result.find('dl').find('dt').findAll('strong')
            if len(result_name) < 1:
                break
            elif re.search('\d', row['title']) != None and len(result_name) == 1:
                movie_name = result_name[0].text
            elif re.search('\d', row['title']) != None:
                movie_name = result_name[1].text
            else:
                movie_name = result_name[0].text
            movie_year = result.find('dl').find('dd', {'class':'etc'}).text[-4:]
            try:
                if movie_name.lower() == row['title'].lower() and int(movie_year) == row['year']:
                    movie_url = result.find('dl').find('dt').find('a').get('href')
                    movie_info.loc[idx, 'movie_url'] = movie_url
                    print(movie_name, movie_url)
                    break
            except ValueError:
                continue

In [ ]:
# 영화 줄거리 return 하는 함수
import re
import requests
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from pprint import pprint
import pandas as pd

movie_info = pd.read_csv('movie_info.csv')
movie_info.head()

def return_synopsis(id):
    row = movie_info[movie_info['id'] == id]
    movie_url = row['movie_url'].values[0]
    if movie_url == '0':
        return "movie_url이 없습니다" # movie_url이 없을 때 return 값
    try:
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('https://', adapter)

        url = f"https://movie.naver.com{movie_url}"
        html = session.get(url, timeout=120).text
    except HTTPError as e:
        pass
    else:
        soup = BeautifulSoup(html, 'html.parser')
        result = soup.find('p', {'class':'con_tx'}).text.replace('\r\xa0', '<br>')
        return result
    
start = time.time()
print(return_synopsis(12))
end = time.time()

print(f"{end - start:.5f}")